In [1]:

import numpy as np
import pandas as pd

import os
from operator import itemgetter
import copy
import time

import torch
import torchvision
import torchvision.transforms as transform
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, ConcatDataset
import torch.nn as nn
import torchvision.models as models
from torchvision.utils import make_grid
import torch.nn.functional as F
import torch.optim as optim
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix, classification_report

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.image import imread
import seaborn as sns
import random as rn
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')



In [2]:

def accuracy(loader, model):
    """Calculate the accuracy of a given model on a given loader"""
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in loader:
            images, labels = data
            # calculate outputs by running images through the network
            outs = model(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total


In [3]:

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64,128,5)
        self.conv4 = nn.Conv2d(128,128,5)
        self.fc1 = nn.Linear(5*5*128, 120)
        # self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(120, 5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [4]:
path = '../input/flowers-recognition/flowers/'

In [5]:
data = ImageFolder(path, transform=transform.Compose([
                                 transform.Resize((150, 150)),
                                 transform.ToTensor(),
                                 transform.Normalize((0.4124234616756439, 0.3674212694168091, 0.2578217089176178),
                                                     (0.3268945515155792, 0.29282665252685547, 0.29053378105163574))]))
invTrans = transform.Compose([ transform.Normalize(mean = [ 0., 0., 0. ],
                                                     std = [ 1/0.3268945515155792, 1/0.29282665252685547, 1/0.29053378105163574 ]),
                                transform.Normalize(mean = [ -0.4124234616756439, -0.3674212694168091, -0.2578217089176178 ],
                                                     std = [ 1., 1., 1. ])])



In [7]:
aug1 = transform.Compose([transform.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
                          transform.RandomRotation(5),
                          transform.RandomAffine(degrees=11, translate=(0.1,0.1), scale=(0.8,0.8))])
aug2 = transform.Compose([transform.RandomHorizontalFlip(),
                         transform.RandomRotation(10),
                         transform.RandomAffine(translate=(0.05,0.05), degrees=0),
                         transform.RandomErasing(inplace=True, scale=(0.01, 0.23))])
aug3 = transform.Compose([transform.RandomHorizontalFlip(p=0.5),
                          transform.RandomRotation(15),
                          transform.RandomAffine(translate=(0.08,0.1), degrees=15)])

In [24]:
aug_data = ImageFolder(path, transform=transform.Compose([transform.Resize((150, 150)),
                                                          transform.ToTensor(),
                                                          transform.Normalize((0.4124234616756439, 0.3674212694168091, 0.2578217089176178),
                                                                               (0.3268945515155792, 0.29282665252685547, 0.29053378105163574)),
                                                          transform.RandomHorizontalFlip(),
                                                          transform.RandomRotation(10),
                                                          transform.RandomAffine(translate=(0.05,0.05), degrees=0),
                                                          transform.RandomErasing(inplace=True, scale=(0.01, 0.23))]))

In [6]:

batch_size = 120
seed = 42
train, test = train_test_split(data, test_size=0.2, shuffle=True, random_state=seed)


In [28]:
big_train = ConcatDataset([train, aug_data])

In [31]:
trainloader = torch.utils.data.DataLoader(big_train, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

aug_trainloader = torch.utils.data.DataLoader(aug_data, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(test, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('Daisy', 'Dandelion', 'Rose', 'Sunflower', 'Tulip')


In [32]:

fig,ax=plt.subplots(2,3)
fig.set_size_inches(15,10)
img,y = iter(trainloader).next()
img = invTrans(img)
for i in range(2):
    for j in range (3):
        l=rn.randint(0,batch_size-1)
        # label = list(train_dataset.class_indices.keys())[np.argmax(y[l])]
        npimg = img[l].numpy()
        ax[i,j].imshow(np.transpose(npimg, (1, 2, 0)))
        ax[i,j].set_title(classes[y[l]])

plt.tight_layout()


In [27]:

fig,ax=plt.subplots(2,3)
fig.set_size_inches(15,10)
img,y = iter(aug_trainloader).next()
img = invTrans(img)
for i in range(2):
    for j in range (3):
        l=rn.randint(0,batch_size-1)
        # label = list(train_dataset.class_indices.keys())[np.argmax(y[l])]
        npimg = img[l].numpy()
        ax[i,j].imshow(np.transpose(npimg, (1, 2, 0)))
        ax[i,j].set_title(classes[y[l]])

plt.tight_layout()


In [33]:
net = Net()

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [35]:

train_acc = []
test_acc = []
epochs = 15
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        # print(i)
#         if i % batch_size == 31:    # print every mini-batches
    print(f'[Epoch: {epoch + 1}, {i + 1:5d}] loss: {running_loss / (i+1):.3f}')
#             running_loss = 0.0
    train_acc.append(accuracy(trainloader, net))
    test_acc.append(accuracy(testloader, net))
print('Finished Training')
plt.figure(figsize=(4, 8))
# plt.subplot(1, 2, 1)
plt.plot(range(1,epochs+1), train_acc, label='Training Accuracy')
plt.plot(range(1,epochs+1), test_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylim(0.4,1.0)
plt.xlim(0,epochs+1)
plt.title('Training and Validation Accuracy')
plt.show()


In [36]:
plt.figure(figsize=(5, 8))
plt.plot(range(1,epochs+1), train_acc, label='Training Accuracy')
plt.plot(range(1,epochs+1), test_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylim(0.4,1.0)
plt.xlim(1,epochs+1)
plt.title('Training and Validation Accuracy')
plt.show()

In [39]:
fig,ax=plt.subplots(2,3)
fig.set_size_inches(15,10)
img,y = iter(testloader).next()
outputs = net(img)
_, pred= torch.max(outputs, 1)
img = invTrans(img)
for i in range(2):
    for j in range (3):
        l=rn.randint(0,batch_size-1)
        # label = list(train_dataset.class_indices.keys())[np.argmax(y[l])]
        npimg = img[l].numpy()
        ax[i,j].imshow(np.transpose(npimg, (1, 2, 0)))
        if y[l] == pred[l]:
            ax[i,j].set_title('Actual: '+classes[y[l]]+'  Predicted: '+classes[pred[l]], color = 'green')
        else:
            ax[i,j].set_title('Actual: '+classes[y[l]]+'  Predicted: '+classes[pred[l]], color = 'red')

plt.tight_layout()
# print images

In [40]:
test_acc

In [3]:
device= torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
device